## 다중 회귀 연습문제
-------
  회귀 문제를 풀때 분류 문제와 가장 큰 차이점은 모델과 평가지표가 다르다는 것이다.
  예를 들어 분류에서 RandomForestClassifier을 사용한다면 회귀에서는 RandomForestRegressor을 사용한다.
  간혹 분류 모델을 사용해 예측값 성능이 현저히 떨어져 0점 처리되는 경우가 있다.

  따라서 문제에서 필요한 지표가 사이킷런에 포함되어 있지 않다면 직접 구현해야 할 수도 있다. 이 경우에는 알고 있는 회귀 평가 지표를 사용해 문제를 해결하는 것이 좋다.

### 항공권 가격 예측
- 항공권 티켓 가격을 예측하시오
  - 제공된 데이터 목록 : flight_train.csv , flight_test.csv
  - 예측할 컬럼 : price

- 성능 평가 : RMSE

In [75]:
from google.colab import files
uploaded = files.upload()

Saving flight_train.csv to flight_train.csv
Saving flight_test.csv to flight_test.csv


In [113]:
import pandas as pd
train = pd.read_csv("flight_train.csv")
test = pd.read_csv("flight_test.csv")
train.shape, test.shape

((10505, 11), (4502, 10))

In [114]:
target = train.pop('price')

In [82]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10505 entries, 0 to 10504
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           10505 non-null  object 
 1   flight            10505 non-null  object 
 2   source_city       10505 non-null  object 
 3   departure_time    10505 non-null  object 
 4   stops             10505 non-null  object 
 5   arrival_time      10505 non-null  object 
 6   destination_city  10505 non-null  object 
 7   class             10505 non-null  object 
 8   duration          10505 non-null  float64
 9   days_left         10505 non-null  int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 820.8+ KB


In [84]:
train.isnull().sum()

,0
airline,0
flight,0
source_city,0
departure_time,0
stops,0
arrival_time,0
destination_city,0
class,0
duration,0
days_left,0


In [109]:
# 카테고리 비교
cols = train.select_dtypes(include='object').columns
for col in cols:
  set_train = set(train[col])
  set_test = set(test[col])
  same= set_train == set_test
  if same:
    print(col,"\n 일치한 컬럼")
  else:
    print(col,"\n 불일치 컬럼")

airline 
 일치한 컬럼
flight 
 불일치 컬럼
source_city 
 일치한 컬럼
departure_time 
 일치한 컬럼
stops 
 일치한 컬럼
arrival_time 
 일치한 컬럼
destination_city 
 일치한 컬럼
class 
 일치한 컬럼


In [88]:
train.describe()

,duration,days_left
count,10505.000000,10505.000000
mean,12.225536,26.050547
std,7.182264,13.539947
min,0.830000,1.000000
25%,6.750000,15.000000
50%,11.250000,26.000000
75%,16.170000,38.000000
max,40.500000,49.000000


In [89]:
train.shape,test.shape

((10505, 10), (4502, 10))

In [115]:
train = train.drop("flight",axis=1)
test = test.drop("flight",axis=1)

In [117]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(train,target,test_size=0.2,random_state=0)

print("분할된 데이터 크기")
print(X_tr.shape,X_val.shape,y_tr.shape,y_val.shape)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import mean_squared_error
#RMSE 결과반환
result = mean_squared_error(y_val,pred,squared=False)
print("RMSE : ",result)

pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv",index=False)

print(pd.read_csv('result.csv').head(3))

분할된 데이터 크기
(8404, 37) (2101, 37) (8404,) (2101,)
RMSE :  4376.841613585934
       pred
0  57356.34
1   5334.44
2  13244.83


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### 성능 개선
    flight 컬럼은 포함하되 중복 제거하고 일부만(앞의 영문은 모두 제거) 포함하겠다.

In [124]:
import pandas as pd
train = pd.read_csv("flight_train.csv")
test = pd.read_csv("flight_test.csv")
train.shape, test.shape

((10505, 11), (4502, 10))

In [125]:
target = train.pop('price')

In [120]:
train['flight']

,flight
0,UK-776
1,UK-852
2,6E-2348
3,AI-763
4,6E-752
...,...
10500,UK-864
10501,UK-774
10502,I5-1531
10503,UK-651


In [126]:
train['f2'] = train['flight'].str.split("-").str[1].astype(int)
test['f2'] = test['flight'].str.split("-").str[1].astype(int)

In [127]:
train = train.drop('flight',axis=1)
test = test.drop('flight',axis=1)

In [129]:
# 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols = ['duration','days_left']
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

# 원-핫 인코딩
train = pd.get_dummies(train)
test= pd.get_dummies(test)

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(train,target,test_size=0.2,random_state=0)

print("분할된 데이터 크기")
print(X_tr.shape,X_val.shape,y_tr.shape,y_val.shape)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=20,n_estimators=200,random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import mean_squared_error
#RMSE 결과반환
result = mean_squared_error(y_val,pred,squared=False)
print("RMSE : ",result)



분할된 데이터 크기
(8404, 38) (2101, 38) (8404,) (2101,)
RMSE :  3675.155093297134


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
